In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.7 MB/s eta 0:00:00


In [ ]:
from itertools import product
from Bio import SeqIO
import pandas as pd

# Define a function to compute TPC
def compute_tpc(sequence):
    tripeptides = [''.join(p) for p in product("ACDEFGHIKLMNPQRSTVWY", repeat=3)]
    tpc_counts = {tripeptide: 0 for tripeptide in tripeptides}

    for i in range(len(sequence) - 2):
        tripeptide = sequence[i:i+3]
        if tripeptide in tpc_counts:
            tpc_counts[tripeptide] += 1

    total_tripeptides = sum(tpc_counts.values())
    if total_tripeptides > 0:
        tpc_counts = {key: val / total_tripeptides for key, val in tpc_counts.items()}

    return tpc_counts

# Process FASTA files
def process_fasta(input_path, output_path):
    data = []
    for record in SeqIO.parse(input_path, "fasta"):
        tpc_features = compute_tpc(str(record.seq))
        data.append(tpc_features)  # Only append TPC features, no IDs

    df = pd.DataFrame(data)
    df.to_csv(output_path, index=False)

# Paths
main_p = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/10_TPC (Tripeptide Composition)/POSITIVE_main (2) (1).fasta"
main_n = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/10_TPC (Tripeptide Composition)/NEGATIVE_main (2) (1).fasta"
validation_p = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/10_TPC (Tripeptide Composition)/POSITIVE_validation (2) (1).fasta"
validation_n = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/10_TPC (Tripeptide Composition)/NEGATIVE_validation (2) (1).fasta"

# Output paths
output_main_p = "/content/positive_main_tpc.csv"
output_main_n = "/content/negative_main_tpc.csv"
output_validation_p = "/content/positive_validation_tpc.csv"
output_validation_n = "/content/negative_validation_tpc.csv"

# Process and save
process_fasta(main_p, output_main_p)
process_fasta(main_n, output_main_n)
process_fasta(validation_p, output_validation_p)
process_fasta(validation_n, output_validation_n)


# **All Model **

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/10_TPC (Tripeptide Composition)/positive_main_tpc.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/10_TPC (Tripeptide Composition)/negative_main_tpc (1).csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/10_TPC (Tripeptide Composition)/positive_validation_tpc.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/10_TPC (Tripeptide Composition)/negative_validation_tpc.csv")

In [ ]:
# Label the datasets
main_p['label'] = 1
main_n['label'] = 0
validation_p['label'] = 1
validation_n['label'] = 0

# Combine datasets
train_data = pd.concat([main_p, main_n], ignore_index=True)
val_data = pd.concat([validation_p, validation_n], ignore_index=True)

# Separate features and labels
X_train = train_data.drop(columns=['label']).values
y_train = train_data['label'].values
X_val = val_data.drop(columns=['label']).values
y_val = val_data['label'].values


In [ ]:
# Dictionary of models
models = {
    "SVM": SVC(kernel='linear', probability=True),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
    "Neural Network": Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ]),
    "MLP": Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the neural network models
models["Neural Network"].compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
models["MLP"].compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Store accuracies
results = []

# Train each model and evaluate
for name, model in models.items():
    print(f"\nTraining {name}...")

    if name in ["Neural Network", "MLP"]:
        # Neural Network training
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), verbose=0)
        train_pred = (model.predict(X_train) > 0.5).astype("int32")
        val_pred = (model.predict(X_val) > 0.5).astype("int32")
    else:
        # Traditional ML model training
        model.fit(X_train, y_train)
        train_pred = model.predict(X_train)
        val_pred = model.predict(X_val)

    # Calculate train and validation accuracy
    train_accuracy = accuracy_score(y_train, train_pred)
    val_accuracy = accuracy_score(y_val, val_pred)

    results.append({"Model": name, "Train Accuracy": train_accuracy, "Validation Accuracy": val_accuracy})


Training SVM...

Training Decision Tree...

Training Random Forest...

Training Logistic Regression...

Training k-NN...

Training Naive Bayes...

Training Gradient Boosting...

Training XGBoost...


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:59:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001556 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 890
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 116
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Training Neural Network...
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

Training MLP...
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [ ]:
# Convert to DataFrame and sort by Validation Accuracy and Train Accuracy in descending order
results_df = pd.DataFrame(results).sort_values(by=["Validation Accuracy", "Train Accuracy"], ascending=False).reset_index(drop=True)

# Display results
print("\nModel Accuracy Table (Descending Order of Validation Accuracy)")
print(results_df)


Model Accuracy Table (Descending Order of Validation Accuracy)
                  Model  Train Accuracy  Validation Accuracy
0         Random Forest        0.995704             0.926667
1         Decision Tree        0.995704             0.913333
2        Neural Network        0.995704             0.906667
3                   MLP        0.995704             0.903333
4              CatBoost        0.970790             0.900000
5               XGBoost        0.963918             0.896667
6              AdaBoost        0.939863             0.890000
7     Gradient Boosting        0.939863             0.876667
8           Naive Bayes        0.959622             0.866667
9                  k-NN        0.928694             0.856667
10             LightGBM        0.887457             0.840000
11  Logistic Regression        0.849656             0.823333
12                  SVM        0.809278             0.773333


# **CROSS VALIDATION**

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
# Dictionary of models
models = {
    "SVM": SVC(kernel='linear', probability=True),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
}


In [ ]:
# Define Neural Network models
def create_neural_network(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_mlp(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [ ]:
# Cross-validation for traditional models
results = []

# For traditional ML models, we use cross_val_score
for name, model in models.items():
    print(f"\nPerforming Cross-validation for {name}...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Calculate cross-validation accuracy
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    mean_accuracy = np.mean(cv_scores)
    std_accuracy = np.std(cv_scores)

    results.append({"Model": name, "Mean CV Accuracy": mean_accuracy, "STD CV Accuracy": std_accuracy})

# Cross-validation for Neural Networks (manual implementation)
for name, create_model in [("Neural Network", create_neural_network), ("MLP", create_mlp)]:
    print(f"\nPerforming Cross-validation for {name}...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Custom function to calculate accuracy for neural networks
    def neural_network_cross_val(model_func, X_train, y_train):
        accuracies = []
        for train_index, val_index in cv.split(X_train, y_train):
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

            model = model_func(X_train.shape[1])  # Create a new model for each fold
            model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=32, verbose=0)

            y_pred = (model.predict(X_val_fold) > 0.5).astype("int32")
            accuracy = accuracy_score(y_val_fold, y_pred)
            accuracies.append(accuracy)

        return np.mean(accuracies), np.std(accuracies)

    mean_accuracy, std_accuracy = neural_network_cross_val(create_model, X_train, y_train)
    results.append({"Model": name, "Mean CV Accuracy": mean_accuracy, "STD CV Accuracy": std_accuracy})



Performing Cross-validation for SVM...

Performing Cross-validation for Decision Tree...

Performing Cross-validation for Random Forest...

Performing Cross-validation for Logistic Regression...

Performing Cross-validation for k-NN...

Performing Cross-validation for Naive Bayes...

Performing Cross-validation for Gradient Boosting...

Performing Cross-validation for XGBoost...


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:06:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:06:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:07:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:07:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:07:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e


Performing Cross-validation for LightGBM...
[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001110 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 631
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 81
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1


Performing Cross-validation for Neural Network...


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [ ]:
# Convert to DataFrame and display
cv_results_df = pd.DataFrame(results).sort_values(by="Mean CV Accuracy", ascending=False).reset_index(drop=True)
print("\nCross-Validation Accuracy Table")
print(cv_results_df)


Cross-Validation Accuracy Table
                  Model  Mean CV Accuracy  STD CV Accuracy
0                   MLP          0.932992         0.004345
1        Neural Network          0.925259         0.008398
2         Random Forest          0.919239         0.011677
3              CatBoost          0.898613         0.018209
4           Naive Bayes          0.892648         0.025944
5               XGBoost          0.888320         0.013813
6         Decision Tree          0.869406         0.018605
7     Gradient Boosting          0.868555         0.017520
8              AdaBoost          0.866845         0.015296
9                  k-NN          0.856504         0.022010
10  Logistic Regression          0.804976         0.016045
11             LightGBM          0.784372         0.013287
12                  SVM          0.766320         0.012068


# **Hyperparameter optimization with Optuna**

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install scikeras

In [ ]:
import pandas as pd
import optuna
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/positive_main_tpc.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/negative_main_tpc (1).csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/positive_validation_tpc.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/negative_validation_tpc.csv")

# Combine positive and negative datasets
main_data = pd.concat([main_p.assign(label=1), main_n.assign(label=0)])
validation_data = pd.concat([validation_p.assign(label=1), validation_n.assign(label=0)])

# Split features and labels
X_train = main_data.drop("label", axis=1)
y_train = main_data["label"]
X_val = validation_data.drop("label", axis=1)
y_val = validation_data["label"]



# Define models with MLP included
models = {
    "SVM": lambda trial: SVC(
        C=trial.suggest_float("C", 0.1, 10.0),
        kernel=trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
    ),
    "Decision Tree": lambda trial: DecisionTreeClassifier(
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Random Forest": lambda trial: RandomForestClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Logistic Regression": lambda trial: LogisticRegression(
        C=trial.suggest_float("C", 0.1, 10.0),
        solver=trial.suggest_categorical("solver", ["lbfgs", "liblinear"])
    ),
    "k-NN": lambda trial: KNeighborsClassifier(
        n_neighbors=trial.suggest_int("n_neighbors", 3, 20)
    ),
    "Naive Bayes": lambda trial: GaussianNB(),
    "Gradient Boosting": lambda trial: GradientBoostingClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        max_depth=trial.suggest_int("max_depth", 3, 20)
    ),
    "XGBoost": lambda trial: XGBClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        use_label_encoder=False,
        eval_metric="logloss"
    ),
    "LightGBM": lambda trial: LGBMClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5)
    ),
    "AdaBoost": lambda trial: AdaBoostClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 1.0)
    ),
    "Neural Network": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("hidden_layer_1", 10, 100),
            trial.suggest_int("hidden_layer_2", 10, 100)
        ),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200
    ),
    "MLP": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("layer_1", 50, 150),
            trial.suggest_int("layer_2", 50, 150)
        ),
        activation=trial.suggest_categorical("activation", ["logistic", "tanh", "relu"]),
        solver=trial.suggest_categorical("solver", ["adam", "sgd"]),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200,
        random_state=42
    )
}


results = []

def optimize_model(model_name, model_func):
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        return accuracy_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    # Store the results
    results.append({
        "Model": model_name,
        "Accuracy": study.best_value,
        "Best Params": study.best_params
    })

# Run optimization for all models
for model_name, model_func in models.items():
    print(f"Optimizing {model_name}...")
    optimize_model(model_name, model_func)


# Convert results to a DataFrame
results_df = pd.DataFrame(results)


# Display the DataFrame
print(results_df)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
[I 2025-01-09 12:44:45,792] A new study created in memory with name: no-name-ae6c1430-21bc-496f-a97e-12f78d739c7b


Optimizing SVM...


[I 2025-01-09 12:44:56,357] Trial 0 finished with value: 0.9033333333333333 and parameters: {'C': 5.157137305824213, 'kernel': 'rbf'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 12:45:01,139] Trial 1 finished with value: 0.8766666666666667 and parameters: {'C': 5.440333510640703, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 12:45:08,342] Trial 2 finished with value: 0.8966666666666666 and parameters: {'C': 6.586790435439739, 'kernel': 'linear'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 12:45:17,761] Trial 3 finished with value: 0.9033333333333333 and parameters: {'C': 2.568006909352526, 'kernel': 'rbf'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 12:45:22,999] Trial 4 finished with value: 0.8866666666666667 and parameters: {'C': 6.06768284122189, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 12:45:27,593] Trial 5 finished with value: 0.8866666666666667

Optimizing Decision Tree...


[I 2025-01-09 12:49:17,848] Trial 0 finished with value: 0.7466666666666667 and parameters: {'max_depth': 5, 'min_samples_split': 2}. Best is trial 0 with value: 0.7466666666666667.
[I 2025-01-09 12:49:18,136] Trial 1 finished with value: 0.81 and parameters: {'max_depth': 11, 'min_samples_split': 2}. Best is trial 1 with value: 0.81.
[I 2025-01-09 12:49:18,472] Trial 2 finished with value: 0.8533333333333334 and parameters: {'max_depth': 18, 'min_samples_split': 2}. Best is trial 2 with value: 0.8533333333333334.
[I 2025-01-09 12:49:18,680] Trial 3 finished with value: 0.7366666666666667 and parameters: {'max_depth': 5, 'min_samples_split': 3}. Best is trial 2 with value: 0.8533333333333334.
[I 2025-01-09 12:49:18,925] Trial 4 finished with value: 0.7933333333333333 and parameters: {'max_depth': 9, 'min_samples_split': 4}. Best is trial 2 with value: 0.8533333333333334.
[I 2025-01-09 12:49:19,209] Trial 5 finished with value: 0.7466666666666667 and parameters: {'max_depth': 5, 'min_sa

...

Optimizing Neural Network...


[I 2025-01-09 13:37:14,582] Trial 0 finished with value: 0.8966666666666666 and parameters: {'hidden_layer_1': 47, 'hidden_layer_2': 90, 'learning_rate_init': 0.010882835768780751}. Best is trial 0 with value: 0.8966666666666666.
[I 2025-01-09 13:37:29,317] Trial 1 finished with value: 0.9033333333333333 and parameters: {'hidden_layer_1': 11, 'hidden_layer_2': 36, 'learning_rate_init': 0.0597010622485078}. Best is trial 1 with value: 0.9033333333333333.
[I 2025-01-09 13:37:50,023] Trial 2 finished with value: 0.9 and parameters: {'hidden_layer_1': 51, 'hidden_layer_2': 54, 'learning_rate_init': 0.07957555377787122}. Best is trial 1 with value: 0.9033333333333333.
[I 2025-01-09 13:38:05,934] Trial 3 finished with value: 0.9066666666666666 and parameters: {'hidden_layer_1': 49, 'hidden_layer_2': 27, 'learning_rate_init': 0.04404997770702086}. Best is trial 3 with value: 0.9066666666666666.
[I 2025-01-09 13:38:30,090] Trial 4 finished with value: 0.9066666666666666 and parameters: {'hidde

Optimizing MLP...


[I 2025-01-09 13:49:22,019] Trial 0 finished with value: 0.9166666666666666 and parameters: {'layer_1': 51, 'layer_2': 58, 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.09490444687482802}. Best is trial 0 with value: 0.9166666666666666.
[I 2025-01-09 13:49:29,030] Trial 1 finished with value: 0.5 and parameters: {'layer_1': 92, 'layer_2': 140, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.015123139097679233}. Best is trial 0 with value: 0.9166666666666666.
[I 2025-01-09 13:49:43,129] Trial 2 finished with value: 0.5 and parameters: {'layer_1': 142, 'layer_2': 50, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.06833250810577617}. Best is trial 0 with value: 0.9166666666666666.
[I 2025-01-09 13:49:52,621] Trial 3 finished with value: 0.5 and parameters: {'layer_1': 87, 'layer_2': 58, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.08128143091345429}. Best is trial 0 with value: 0.9166666666666666.
[I 2025-01-09 

                  Model  Accuracy  \
0                   SVM  0.906667   
1         Decision Tree  0.866667   
2         Random Forest  0.893333   
3   Logistic Regression  0.893333   
4                  k-NN  0.883333   
5           Naive Bayes  0.866667   
6     Gradient Boosting  0.900000   
7               XGBoost  0.913333   
8              LightGBM  0.853333   
9              AdaBoost  0.886667   
10       Neural Network  0.920000   
11                  MLP  0.920000   

                                          Best Params  
0          {'C': 1.4844504033388803, 'kernel': 'rbf'}  
1           {'max_depth': 20, 'min_samples_split': 7}  
2   {'n_estimators': 61, 'max_depth': 20, 'min_sam...  
3         {'C': 9.988533805424675, 'solver': 'lbfgs'}  
4                                  {'n_neighbors': 3}  
5                                                  {}  
6   {'n_estimators': 96, 'learning_rate': 0.263784...  
7   {'n_estimators': 255, 'max_depth': 11, 'learni...  
8   {'n_estima

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score, roc_auc_score # Import confusion_matrix and other metrics

# Function to calculate metrics with model name
def calculate_metrics(y_true, y_pred, model_name=None):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    mcc = matthews_corrcoef(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    return {
        "Model": model_name,
        "Accuracy": accuracy,
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "MCC": mcc,
        "Kappa": kappa,
        "AUC": auc,
    }

# Results storage
results = []

# Optimization function
def optimize_model_with_metrics(model_name, model_func):
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        trial.set_user_attr("preds", preds)
        metrics = calculate_metrics(y_val, preds, model_name=model_name)
        return metrics["Accuracy"]

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    # Store the best trial metrics
    best_trial_preds = study.best_trial.user_attrs["preds"]
    best_metrics = calculate_metrics(y_val, best_trial_preds, model_name=model_name)
    best_metrics["Best Params"] = study.best_params
    results.append(best_metrics)

# Run optimization for all models
for model_name, model_func in models.items():
    print(f"Optimizing {model_name}...")
    optimize_model_with_metrics(model_name, model_func)

# Store the final results with selected metrics and best parameters
final_results = []
for result in results:
    final_results.append(
        {
            "Model": result["Model"],
            "Accuracy": result["Accuracy"],
            "Sensitivity": result["Sensitivity"],
            "Specificity": result["Specificity"],
            "MCC": result["MCC"],
            "Kappa": result["Kappa"],
            "AUC": result["AUC"],
            "Best Params": result["Best Params"],
        }
    )

# Convert results to a DataFrame and display it
final_results_df = pd.DataFrame(final_results)
print(final_results_df)


[I 2025-01-09 14:04:56,893] A new study created in memory with name: no-name-7bea2d43-44d0-4544-aa68-4641afb0ce71


Optimizing SVM...


[I 2025-01-09 14:05:02,800] Trial 0 finished with value: 0.8633333333333333 and parameters: {'C': 3.464289084451455, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.8633333333333333.
[I 2025-01-09 14:05:11,647] Trial 1 finished with value: 0.8366666666666667 and parameters: {'C': 1.4412721924465302, 'kernel': 'linear'}. Best is trial 0 with value: 0.8633333333333333.
[I 2025-01-09 14:05:21,775] Trial 2 finished with value: 0.9033333333333333 and parameters: {'C': 6.703779296188013, 'kernel': 'rbf'}. Best is trial 2 with value: 0.9033333333333333.
[I 2025-01-09 14:05:26,697] Trial 3 finished with value: 0.8666666666666667 and parameters: {'C': 4.364403984502521, 'kernel': 'sigmoid'}. Best is trial 2 with value: 0.9033333333333333.
[I 2025-01-09 14:05:36,266] Trial 4 finished with value: 0.79 and parameters: {'C': 1.9154649505584882, 'kernel': 'poly'}. Best is trial 2 with value: 0.9033333333333333.
[I 2025-01-09 14:05:42,417] Trial 5 finished with value: 0.8833333333333333 and param

Optimizing Decision Tree...


[I 2025-01-09 14:09:20,992] Trial 0 finished with value: 0.81 and parameters: {'max_depth': 11, 'min_samples_split': 5}. Best is trial 0 with value: 0.81.
[I 2025-01-09 14:09:21,245] Trial 1 finished with value: 0.7933333333333333 and parameters: {'max_depth': 9, 'min_samples_split': 9}. Best is trial 0 with value: 0.81.
[I 2025-01-09 14:09:21,592] Trial 2 finished with value: 0.8566666666666667 and parameters: {'max_depth': 17, 'min_samples_split': 7}. Best is trial 2 with value: 0.8566666666666667.
[I 2025-01-09 14:09:21,827] Trial 3 finished with value: 0.7833333333333333 and parameters: {'max_depth': 8, 'min_samples_split': 7}. Best is trial 2 with value: 0.8566666666666667.
[I 2025-01-09 14:09:22,065] Trial 4 finished with value: 0.7866666666666666 and parameters: {'max_depth': 8, 'min_samples_split': 9}. Best is trial 2 with value: 0.8566666666666667.
[I 2025-01-09 14:09:22,358] Trial 5 finished with value: 0.8266666666666667 and parameters: {'max_depth': 13, 'min_samples_split':

...

Optimizing Neural Network...


[I 2025-01-09 15:13:28,772] Trial 0 finished with value: 0.9 and parameters: {'hidden_layer_1': 32, 'hidden_layer_2': 93, 'learning_rate_init': 0.013267786891709595}. Best is trial 0 with value: 0.9.
[I 2025-01-09 15:13:54,301] Trial 1 finished with value: 0.9033333333333333 and parameters: {'hidden_layer_1': 100, 'hidden_layer_2': 52, 'learning_rate_init': 0.09535007404827542}. Best is trial 1 with value: 0.9033333333333333.
[I 2025-01-09 15:14:02,192] Trial 2 finished with value: 0.9 and parameters: {'hidden_layer_1': 26, 'hidden_layer_2': 64, 'learning_rate_init': 0.027798667020607522}. Best is trial 1 with value: 0.9033333333333333.
[I 2025-01-09 15:14:10,493] Trial 3 finished with value: 0.8966666666666666 and parameters: {'hidden_layer_1': 32, 'hidden_layer_2': 12, 'learning_rate_init': 0.044188343438543914}. Best is trial 1 with value: 0.9033333333333333.
[I 2025-01-09 15:14:26,522] Trial 4 finished with value: 0.9 and parameters: {'hidden_layer_1': 74, 'hidden_layer_2': 19, 'le

Optimizing MLP...


[I 2025-01-09 15:26:03,005] Trial 0 finished with value: 0.5 and parameters: {'layer_1': 138, 'layer_2': 76, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.0745826634625593}. Best is trial 0 with value: 0.5.
[I 2025-01-09 15:26:45,165] Trial 1 finished with value: 0.9233333333333333 and parameters: {'layer_1': 74, 'layer_2': 135, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.08909100795993583}. Best is trial 1 with value: 0.9233333333333333.
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-09 15:28:29,774] Trial 2 finished with value: 0.9066666666666666 and parameters: {'layer_1': 116, 'layer_2': 139, 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.011411535215905957}. Best is trial 1 with value: 0.9233333333333333.
[I 2025-01-09 15:28:35,128] Tr

                  Model  Accuracy  Sensitivity  Specificity       MCC  \
0                   SVM  0.903333     0.873333     0.933333  0.808123   
1         Decision Tree  0.873333     0.780000     0.966667  0.760025   
2         Random Forest  0.886667     0.820000     0.953333  0.780300   
3   Logistic Regression  0.893333     0.853333     0.933333  0.789196   
4                  k-NN  0.883333     0.866667     0.900000  0.767093   
5           Naive Bayes  0.866667     0.880000     0.853333  0.733594   
6     Gradient Boosting  0.893333     0.873333     0.913333  0.787297   
7               XGBoost  0.910000     0.886667     0.933333  0.820894   
8              LightGBM  0.856667     0.786667     0.926667  0.720428   
9              AdaBoost  0.860000     0.760000     0.960000  0.734847   
10       Neural Network  0.926667     0.940000     0.913333  0.853637   
11                  MLP  0.926667     0.933333     0.920000  0.853409   

       Kappa       AUC                            

In [ ]:
import pandas as pd
import optuna
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Define models
models = {
    "SVM": lambda trial: SVC(probability=True, C=trial.suggest_float("C", 0.1, 10.0), kernel=trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])),
    "Decision Tree": lambda trial: DecisionTreeClassifier(max_depth=trial.suggest_int("max_depth", 3, 20), min_samples_split=trial.suggest_int("min_samples_split", 2, 10)),
    "Random Forest": lambda trial: RandomForestClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), max_depth=trial.suggest_int("max_depth", 3, 20), min_samples_split=trial.suggest_int("min_samples_split", 2, 10)),
    "Logistic Regression": lambda trial: LogisticRegression(C=trial.suggest_float("C", 0.1, 10.0), solver=trial.suggest_categorical("solver", ["lbfgs", "liblinear"])),
    "k-NN": lambda trial: KNeighborsClassifier(n_neighbors=trial.suggest_int("n_neighbors", 3, 20)),
    "Naive Bayes": lambda trial: GaussianNB(),
    "Gradient Boosting": lambda trial: GradientBoostingClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5), max_depth=trial.suggest_int("max_depth", 3, 20)),
    "XGBoost": lambda trial: XGBClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), max_depth=trial.suggest_int("max_depth", 3, 20), learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5), use_label_encoder=False, eval_metric="logloss"),
    "LightGBM": lambda trial: LGBMClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), max_depth=trial.suggest_int("max_depth", 3, 20), learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5)),
    "AdaBoost": lambda trial: AdaBoostClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), learning_rate=trial.suggest_float("learning_rate", 0.01, 1.0)),
    "Neural Network": lambda trial: MLPClassifier(hidden_layer_sizes=(trial.suggest_int("hidden_layer_1", 10, 100), trial.suggest_int("hidden_layer_2", 10, 100)), learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1), max_iter=200),
    "MLP": lambda trial: MLPClassifier(hidden_layer_sizes=(trial.suggest_int("layer_1", 50, 150), trial.suggest_int("layer_2", 50, 150)), activation=trial.suggest_categorical("activation", ["logistic", "tanh", "relu"]), solver=trial.suggest_categorical("solver", ["adam", "sgd"]), learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1), max_iter=200, random_state=42)
}

# Prepare a dictionary to store model probabilities horizontally
probabilities = {"Target": y_val}  # Starting with the target column (y_val)

# Run optimization and compute probabilities for each model
for model_name, model_func in models.items():
    print(f"Optimizing {model_name}...")
    study = optuna.create_study(direction="maximize")

    # Objective function for Optuna
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        return accuracy_score(y_val, model.predict(X_val))

    study.optimize(objective, n_trials=30)

    # Train the best model using the best hyperparameters
    best_model = model_func(study.best_trial)
    best_model.fit(X_train, y_train)

    # Get predicted probabilities for the positive class (class 1)
    probs = best_model.predict_proba(X_val)[:, 1]

    # Add to the probabilities dictionary
    probabilities[model_name] = probs

# Convert the probabilities dictionary to a DataFrame
probability_df = pd.DataFrame(probabilities)

# Save the probability dataset to a CSV file
probability_df.to_csv("N_TPC_OPTUNA_probability_predictions.csv", index=False)

print("Dataset saved successfully!")


[I 2025-01-09 16:05:36,845] A new study created in memory with name: no-name-20a35274-ccab-43e0-b457-6b7d89e52530


Optimizing SVM...


[I 2025-01-09 16:06:19,886] Trial 0 finished with value: 0.9033333333333333 and parameters: {'C': 6.4235074464357504, 'kernel': 'rbf'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 16:06:40,211] Trial 1 finished with value: 0.8833333333333333 and parameters: {'C': 7.668764259807423, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 16:07:18,772] Trial 2 finished with value: 0.83 and parameters: {'C': 3.7360319976837792, 'kernel': 'poly'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 16:07:49,754] Trial 3 finished with value: 0.8933333333333333 and parameters: {'C': 6.92146927782229, 'kernel': 'linear'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 16:08:25,719] Trial 4 finished with value: 0.87 and parameters: {'C': 2.5342611510835513, 'kernel': 'linear'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-09 16:09:05,330] Trial 5 finished with value: 0.8433333333333334 and parameters: {'C': 1.

Optimizing Decision Tree...


[I 2025-01-09 16:25:43,740] Trial 0 finished with value: 0.78 and parameters: {'max_depth': 7, 'min_samples_split': 5}. Best is trial 0 with value: 0.78.
[I 2025-01-09 16:25:44,051] Trial 1 finished with value: 0.83 and parameters: {'max_depth': 14, 'min_samples_split': 2}. Best is trial 1 with value: 0.83.
[I 2025-01-09 16:25:44,269] Trial 2 finished with value: 0.7766666666666666 and parameters: {'max_depth': 7, 'min_samples_split': 7}. Best is trial 1 with value: 0.83.
[I 2025-01-09 16:25:44,620] Trial 3 finished with value: 0.85 and parameters: {'max_depth': 18, 'min_samples_split': 5}. Best is trial 3 with value: 0.85.
[I 2025-01-09 16:25:44,894] Trial 4 finished with value: 0.81 and parameters: {'max_depth': 11, 'min_samples_split': 8}. Best is trial 3 with value: 0.85.
[I 2025-01-09 16:25:45,133] Trial 5 finished with value: 0.7233333333333334 and parameters: {'max_depth': 4, 'min_samples_split': 10}. Best is trial 3 with value: 0.85.
[I 2025-01-09 16:25:45,707] Trial 6 finished

...

Optimizing Neural Network...


[I 2025-01-09 17:32:47,533] Trial 0 finished with value: 0.92 and parameters: {'hidden_layer_1': 60, 'hidden_layer_2': 22, 'learning_rate_init': 0.04766872956542842}. Best is trial 0 with value: 0.92.
[I 2025-01-09 17:33:08,891] Trial 1 finished with value: 0.92 and parameters: {'hidden_layer_1': 74, 'hidden_layer_2': 59, 'learning_rate_init': 0.09105616871960961}. Best is trial 0 with value: 0.92.
[I 2025-01-09 17:33:57,652] Trial 2 finished with value: 0.8966666666666666 and parameters: {'hidden_layer_1': 54, 'hidden_layer_2': 76, 'learning_rate_init': 0.0015010942069476935}. Best is trial 0 with value: 0.92.
[I 2025-01-09 17:34:14,579] Trial 3 finished with value: 0.91 and parameters: {'hidden_layer_1': 46, 'hidden_layer_2': 50, 'learning_rate_init': 0.06044859810453942}. Best is trial 0 with value: 0.92.
[I 2025-01-09 17:34:32,583] Trial 4 finished with value: 0.91 and parameters: {'hidden_layer_1': 26, 'hidden_layer_2': 30, 'learning_rate_init': 0.07056557670290124}. Best is trial

Optimizing MLP...


[I 2025-01-09 17:45:19,724] Trial 0 finished with value: 0.9166666666666666 and parameters: {'layer_1': 129, 'layer_2': 81, 'activation': 'relu', 'solver': 'adam', 'learning_rate_init': 0.06404304420935956}. Best is trial 0 with value: 0.9166666666666666.
[I 2025-01-09 17:46:02,487] Trial 1 finished with value: 0.91 and parameters: {'layer_1': 109, 'layer_2': 129, 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.054499384177819056}. Best is trial 0 with value: 0.9166666666666666.
[I 2025-01-09 17:46:15,226] Trial 2 finished with value: 0.88 and parameters: {'layer_1': 76, 'layer_2': 79, 'activation': 'logistic', 'solver': 'adam', 'learning_rate_init': 0.09932092910554714}. Best is trial 0 with value: 0.9166666666666666.
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-09 17:47:36,379]

Dataset saved successfully!


Class Feature Vector (CFV)


In [2]:
import optuna
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/positive_main_tpc.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/negative_main_tpc (1).csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/positive_validation_tpc.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/negative_validation_tpc.csv")

# Combine positive and negative datasets
main_data = pd.concat([main_p.assign(label=1), main_n.assign(label=0)])
validation_data = pd.concat([validation_p.assign(label=1), validation_n.assign(label=0)])

# Split features and labels
X_train = main_data.drop("label", axis=1)
y_train = main_data["label"]
X_val = validation_data.drop("label", axis=1)
y_val = validation_data["label"]

# Define models with hyperparameter optimization (Optuna)
models = {
    "SVM": lambda trial: SVC(
         probability=True,
        C=trial.suggest_float("C", 0.1, 10.0),
        kernel=trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
    ),
    "Decision Tree": lambda trial: DecisionTreeClassifier(
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Random Forest": lambda trial: RandomForestClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Logistic Regression": lambda trial: LogisticRegression(
        C=trial.suggest_float("C", 0.1, 10.0),
        solver=trial.suggest_categorical("solver", ["lbfgs", "liblinear"])
    ),
    "k-NN": lambda trial: KNeighborsClassifier(
        n_neighbors=trial.suggest_int("n_neighbors", 3, 20)
    ),
    "Naive Bayes": lambda trial: GaussianNB(),
    "Gradient Boosting": lambda trial: GradientBoostingClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        max_depth=trial.suggest_int("max_depth", 3, 20)
    ),
    "XGBoost": lambda trial: XGBClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        use_label_encoder=False,
        eval_metric="logloss"
    ),
    "LightGBM": lambda trial: LGBMClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5)
    ),
    "AdaBoost": lambda trial: AdaBoostClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 1.0)
    ),
    "Neural Network": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("hidden_layer_1", 10, 100),
            trial.suggest_int("hidden_layer_2", 10, 100)
        ),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200
    ),
    "MLP": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("layer_1", 50, 150),
            trial.suggest_int("layer_2", 50, 150)
        ),
        activation=trial.suggest_categorical("activation", ["logistic", "tanh", "relu"]),
        solver=trial.suggest_categorical("solver", ["adam", "sgd"]),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200,
        random_state=42
    )
}

# Initialize a list to store the CFV data
cfv_data = []

# Define the optimization and prediction function
def optimize_and_predict(model_name, model_func):
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_val)[:, 1]  # Get probability for class 1 (positive)
        return accuracy_score(y_val, model.predict(X_val))

    # Perform optimization with Optuna
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    # Return the best model
    best_model = model_func(study.best_trial)
    best_model.fit(X_train, y_train)
    preds = best_model.predict_proba(X_val)[:, 1]  # Using the probability for class 1 (positive)

    # Append predictions to CFV list
    return preds

# Train each model and generate predictions for CFV
for model_name, model_func in models.items():
    print(f"Training and predicting with {model_name}...")
    preds = optimize_and_predict(model_name, model_func)
    cfv_data.append(preds)

# Convert the CFV data into a DataFrame
cfv_df = pd.DataFrame(np.array(cfv_data).T, columns=models.keys())

# Optionally, add the true labels column
cfv_df["True_Label"] = y_val.values

# Save the CFV dataset to CSV
cfv_df.to_csv("CFV_TPC.csv", index=False)
print("CFV dataset created and saved!")


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
[I 2025-01-16 13:55:11,762] A new study created in memory with name: no-name-d98548f2-1d11-452d-830b-a1fd9464f393


Training and predicting with SVM...


[I 2025-01-16 13:55:47,867] Trial 0 finished with value: 0.81 and parameters: {'C': 2.576322755143849, 'kernel': 'poly'}. Best is trial 0 with value: 0.81.
[I 2025-01-16 13:56:24,424] Trial 1 finished with value: 0.8333333333333334 and parameters: {'C': 4.346971155793399, 'kernel': 'poly'}. Best is trial 1 with value: 0.8333333333333334.
[I 2025-01-16 13:56:53,976] Trial 2 finished with value: 0.8833333333333333 and parameters: {'C': 4.706341750295366, 'kernel': 'linear'}. Best is trial 2 with value: 0.8833333333333333.
[I 2025-01-16 13:57:33,057] Trial 3 finished with value: 0.9033333333333333 and parameters: {'C': 8.446869128095893, 'kernel': 'rbf'}. Best is trial 3 with value: 0.9033333333333333.
[I 2025-01-16 13:58:11,215] Trial 4 finished with value: 0.86 and parameters: {'C': 7.022367754624726, 'kernel': 'poly'}. Best is trial 3 with value: 0.9033333333333333.
[I 2025-01-16 13:58:51,146] Trial 5 finished with value: 0.9033333333333333 and parameters: {'C': 4.739543195512678, 'ker

Training and predicting with Decision Tree...


[I 2025-01-16 14:13:38,150] Trial 0 finished with value: 0.7633333333333333 and parameters: {'max_depth': 6, 'min_samples_split': 9}. Best is trial 0 with value: 0.7633333333333333.
[I 2025-01-16 14:13:38,405] Trial 1 finished with value: 0.7866666666666666 and parameters: {'max_depth': 8, 'min_samples_split': 10}. Best is trial 1 with value: 0.7866666666666666.
[I 2025-01-16 14:13:38,646] Trial 2 finished with value: 0.77 and parameters: {'max_depth': 7, 'min_samples_split': 6}. Best is trial 1 with value: 0.7866666666666666.
[I 2025-01-16 14:13:38,855] Trial 3 finished with value: 0.71 and parameters: {'max_depth': 3, 'min_samples_split': 9}. Best is trial 1 with value: 0.7866666666666666.
[I 2025-01-16 14:13:39,107] Trial 4 finished with value: 0.7866666666666666 and parameters: {'max_depth': 8, 'min_samples_split': 9}. Best is trial 1 with value: 0.7866666666666666.
[I 2025-01-16 14:13:39,418] Trial 5 finished with value: 0.8166666666666667 and parameters: {'max_depth': 13, 'min_sa

...

Training and predicting with Neural Network...


[I 2025-01-16 15:01:11,482] Trial 0 finished with value: 0.9 and parameters: {'hidden_layer_1': 96, 'hidden_layer_2': 95, 'learning_rate_init': 0.011396815284265085}. Best is trial 0 with value: 0.9.
[I 2025-01-16 15:01:22,796] Trial 1 finished with value: 0.8933333333333333 and parameters: {'hidden_layer_1': 50, 'hidden_layer_2': 27, 'learning_rate_init': 0.04772385042498817}. Best is trial 0 with value: 0.9.
[I 2025-01-16 15:01:54,194] Trial 2 finished with value: 0.91 and parameters: {'hidden_layer_1': 84, 'hidden_layer_2': 71, 'learning_rate_init': 0.055484974533201836}. Best is trial 2 with value: 0.91.
[I 2025-01-16 15:02:15,520] Trial 3 finished with value: 0.9066666666666666 and parameters: {'hidden_layer_1': 80, 'hidden_layer_2': 61, 'learning_rate_init': 0.02563372781870428}. Best is trial 2 with value: 0.91.
[I 2025-01-16 15:02:30,928] Trial 4 finished with value: 0.91 and parameters: {'hidden_layer_1': 62, 'hidden_layer_2': 99, 'learning_rate_init': 0.07560629746956768}. Be

Training and predicting with MLP...


[I 2025-01-16 15:11:09,796] Trial 0 finished with value: 0.9133333333333333 and parameters: {'layer_1': 112, 'layer_2': 113, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.058958334853773935}. Best is trial 0 with value: 0.9133333333333333.
[I 2025-01-16 15:11:18,777] Trial 1 finished with value: 0.5 and parameters: {'layer_1': 145, 'layer_2': 66, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.020216416881694073}. Best is trial 0 with value: 0.9133333333333333.
[I 2025-01-16 15:11:45,071] Trial 2 finished with value: 0.89 and parameters: {'layer_1': 93, 'layer_2': 91, 'activation': 'logistic', 'solver': 'adam', 'learning_rate_init': 0.09172591324497353}. Best is trial 0 with value: 0.9133333333333333.
[I 2025-01-16 15:12:35,806] Trial 3 finished with value: 0.9166666666666666 and parameters: {'layer_1': 107, 'layer_2': 113, 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.08274676664271781}. Best is trial 3 with value: 0.91666666666666

CFV dataset created and saved!


CPFV (Combined Probability and Class Feature Vector)

In [3]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

# Load datasets
# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/positive_main_tpc.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/negative_main_tpc (1).csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/positive_validation_tpc.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/negative_validation_tpc.csv")

# Combine positive and negative datasets
main_data = pd.concat([main_p.assign(label=1), main_n.assign(label=0)], ignore_index=True)
validation_data = pd.concat([validation_p.assign(label=1), validation_n.assign(label=0)], ignore_index=True)

# Separate features and labels
X_train = main_data.drop(columns=["label"])
y_train = main_data["label"]
X_val = validation_data.drop(columns=["label"])
y_val = validation_data["label"]

# Initialize models with their tuned hyperparameters
trained_models = {
    "SVM": SVC(C=1.0, kernel="rbf", probability=True),  # Example parameters
    "Decision Tree": DecisionTreeClassifier(max_depth=10, min_samples_split=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5),
    "Logistic Regression": LogisticRegression(C=1.0, solver="lbfgs"),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=10),
    "XGBoost": XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1, use_label_encoder=False, eval_metric="logloss"),
    "LightGBM": LGBMClassifier(n_estimators=100, max_depth=10, learning_rate=0.1),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, learning_rate=0.1),
    "Neural Network (MLPClassifier)": MLPClassifier(hidden_layer_sizes=(100, 50), activation="relu", solver="adam", learning_rate_init=0.01, max_iter=200),
    "Multilayer Perceptron (Custom MLP)": MLPClassifier(hidden_layer_sizes=(128, 64), activation="relu", solver="adam", learning_rate_init=0.01, max_iter=200)
}

# Train all models on the training dataset
for model_name, model in trained_models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)

# Function to create CPFV dataset
def create_cpfv(models, X_data, y_data):
    cpfv_data = pd.DataFrame()
    for model_name, model in models.items():
        # Add predicted class labels
        cpfv_data[f"{model_name}_Class"] = model.predict(X_data)
        # Add predicted probabilities or decision scores
        if hasattr(model, "predict_proba"):
            cpfv_data[f"{model_name}_Prob"] = model.predict_proba(X_data)[:, 1]
        elif hasattr(model, "decision_function"):
            cpfv_data[f"{model_name}_Prob"] = model.decision_function(X_data)
        else:
            cpfv_data[f"{model_name}_Prob"] = cpfv_data[f"{model_name}_Class"]
    # Add true labels
    cpfv_data["True_Label"] = y_data.reset_index(drop=True)
    return cpfv_data

# Create CPFV dataset using validation data
cpfv_dataset = create_cpfv(trained_models, X_val, y_val)

# Save CPFV dataset to CSV
cpfv_dataset.to_csv("CPFV_TPC.csv", index=False)


Training SVM...
Training Decision Tree...
Training Random Forest...
Training Logistic Regression...
Training k-NN...
Training Naive Bayes...
Training Gradient Boosting...
Training XGBoost...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:25:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001353 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 890
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 116
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further